In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import wrds
import datetime
import statsmodels.api as sm

In [2]:
db = wrds.Connection() 
#goqhuB-1hafqe-dojvix

Enter your WRDS username [max]:antb95
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [6]:
sp = db.get_table(library = 'crsp' , table = 'dsp500p')
start = np.where(sp.caldt >= datetime.date(2011,3,17))[0][0]
end = np.where(sp.caldt >= datetime.date(2016,3,17))[0][0]
sp = sp.loc[start:end,['caldt','spindx']]

In [8]:
names = db.get_table(library='crsp', table='stocknames')
names =names.loc[:,['permco' , 'ticker']]

In [9]:
Ticker = ['IBM','MCD','MMM','WMT']
Permco = np.zeros(len(Ticker))
for i in range(len(Ticker)):
    inter = names['permco'].where(names['ticker'] == Ticker[i])
    inter.dropna(inplace = True)
    Permco[i] = inter.iloc[1]

In [10]:
df = pd.DataFrame(index = np.arange(0,len(sp)),columns = ['Date' , 'SPX'] + Ticker)

In [11]:
df.Date = np.array(sp.caldt)
df.SPX = np.array(sp.spindx)
for i in range(len(Ticker)):
    inter = db.raw_sql("select prc, date from crsp.dsf where permco in ("+str(Permco[i])+") and date >='2011-03-17' and date <='2016-03-17'")
    df[Ticker[i]] = inter.prc

In [12]:
df.loc[:,df.columns != 'Date'] = np.log(df.loc[:,df.columns != 'Date']) - np.log(df.loc[:,df.columns != 'Date']).shift(1)
df.dropna(inplace = True)

In [13]:
X = pd.DataFrame({'Ones' : [1] * len(df), 'SPX' : np.array(df.SPX)},index = np.arange(1,len(df)+1,1))
Beta = pd.DataFrame(columns = Ticker,index = ['a' , 'Beta'])
Eps = pd.DataFrame(columns = Ticker, index = df.index)
for i in range(len(Ticker)):
    Y = df[Ticker[i]]
    model = sm.OLS(Y,X).fit()
    Beta.at[:,Ticker[i]] = model.params.to_list()
    Eps.at[:,Ticker[i]] = Y - np.matmul(np.array(X),np.array(Beta[Ticker[i]]))
Beta

,IBM,MCD,MMM,WMT
a,-0.000342,0.000193,0.000121,0.000021
Beta,0.812870,0.583189,0.977735,0.522278


In [14]:
Eps.corr()

,IBM,MCD,MMM,WMT
IBM,1.000000,0.062738,0.001554,-0.003596
MCD,0.062738,1.000000,0.023428,0.121860
MMM,0.001554,0.023428,1.000000,0.029651
WMT,-0.003596,0.121860,0.029651,1.000000


In [15]:
df[Ticker].corr()

,IBM,MCD,MMM,WMT
IBM,1.000000,0.425738,0.528305,0.319660
MCD,0.425738,1.000000,0.497643,0.381854
MMM,0.528305,0.497643,1.000000,0.419516
WMT,0.319660,0.381854,0.419516,1.000000
